In [1]:
import meb
from meb import utils
from meb import datasets
from meb import core
from meb import models

from functools import partial
from typing import List, Tuple

import numpy as np
import pandas as pd
from numba import jit, njit
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import timm
from tqdm import tqdm



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [20]:
class EarlyStoppingValidator(core.MEGCValidator):
    def __init__(self, config: Config, verbose: bool = True):
        super().__init__(config)
        self.verbose = verbose
        self.disable_tqdm = False
        
    def validate_split(self, df: pd.DataFrame, input_data: np.ndarray, labels: np.ndarray, split_name: str):
        """Main setup of each split. Should be called by the overriden validate method."""
        train_data, train_labels, test_data, test_labels = self.split_data(
            df[self.split_column], input_data, labels, split_name
        )
        train_loader = self.get_data_loader(train_data, train_labels, train=True)
        test_loader = self.get_data_loader(test_data, test_labels, train=False)
        self.model = self.cf.model()
        self.criterion = self.cf.criterion()
        self.model.to(self.cf.device)
        self.optimizer = self.cf.optimizer(self.model.parameters())
        self.scheduler = self.cf.scheduler(self.optimizer) if self.cf.scheduler else None
        self.mixup_fn = self.cf.mixup_fn() if self.cf.mixup_fn else None
        
        if self.cf.early_stopping:
            test_metrics, outputs_test = self.train_eval_model(train_loader, test_loader)
            train_metrics = self.evaluate_model(train_loader)
        else:
            self.train_model(train_loader, test_loader)
            train_metrics = self.evaluate_model(train_loader)
            test_metrics, outputs_test = self.evaluate_model(test_loader, test=True)
        return train_metrics, test_metrics, outputs_test
    
    def train_eval_model(
        self,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader
    ) -> Tuple:
        best_f1 = -1
        best_output = torch.tensor([])
        best_epoch = -1
        for epoch in tqdm(range(self.cf.epochs), disable=self.disable_tqdm):
            self.train_one_epoch(epoch, train_loader)
            if self.scheduler:
                self.scheduler.step(epoch + 1)
            # Validate after every epoch
            if self.cf.early_stopping:
                test_metrics, outputs_test = self.evaluate_model(test_loader, test=True)
                epoch_f1 = test_metrics[0]
                if epoch_f1 > best_f1:
                    best_f1 = epoch_f1
                    best_output = outputs_test
                    best_epoch = epoch
        print(f"Best epoch: {best_epoch}")
        return [best_f1], best_output
        
    

In [40]:
c = datasets.MEGC(optical_flow=True, resize=28)
df = c.data_frame
data = c.data

In [39]:
class Config(core.Config):
    action_units = None
    device = torch.device("cuda:1")
    scheduler = None
    num_workers = 0
    criterion = nn.CrossEntropyLoss
    evaluation_fn = utils.MultiClassF1Score
    epochs = 500
    model = partial(models.STSTNet, num_classes=3)
    optimizer = partial(optim.Adam, lr=5e-5, weight_decay=1e-3)
    early_stopping = True

In [12]:
#ststnet
out = EarlyStoppingValidation(Config).validate(df, data)

100%|█████████████████████████████████████████| 100/100 [03:03<00:00,  1.83s/it]


Subject: s1, n=6 | train_mean: 0.9419 | test_mean: 0.4127


100%|█████████████████████████████████████████| 100/100 [03:03<00:00,  1.84s/it]


Subject: s11, n=7 | train_mean: 0.9508 | test_mean: 0.8857


100%|█████████████████████████████████████████| 100/100 [03:06<00:00,  1.87s/it]


...

Subject: 035, n=8 | train_mean: 0.9449 | test_mean: 1.0


100%|█████████████████████████████████████████| 100/100 [02:44<00:00,  1.65s/it]


Subject: 036, n=1 | train_mean: 0.9485 | test_mean: 1.0


100%|█████████████████████████████████████████| 100/100 [02:45<00:00,  1.65s/it]


Subject: 037, n=1 | train_mean: 0.9448 | test_mean: 1.0
Total f1: [0.7127365562380598], SMIC: [0.7139280590438938], CASME2: [0.7035542402774041], SAMM: [0.6618739635157547]


In [41]:
#ststnet early stopping
out = EarlyStoppingValidator(Config).validate(df, data)

100%|█████████████████████████████████████████| 500/500 [01:20<00:00,  6.23it/s]


Best epoch: 9
Subject: s1, n=6 | train_mean: 0.8715 | test_mean: 0.25


100%|█████████████████████████████████████████| 500/500 [01:20<00:00,  6.22it/s]


Best epoch: 442
Subject: s11, n=7 | train_mean: 0.8363 | test_mean: 1.0


100%|█████████████████████████████████████████| 500/500 [01:23<00:00,  6.00it/s]


...

Best epoch: 4
Subject: 035, n=8 | train_mean: 0.839 | test_mean: 1.0


100%|█████████████████████████████████████████| 500/500 [01:20<00:00,  6.19it/s]


Best epoch: 4
Subject: 036, n=1 | train_mean: 0.8599 | test_mean: 1.0


100%|█████████████████████████████████████████| 500/500 [01:20<00:00,  6.23it/s]


Best epoch: 1
Subject: 037, n=1 | train_mean: 0.8723 | test_mean: 1.0
Total f1: [0.7739532794249775], SMIC: [0.7214356452664751], CASME2: [0.8692050483818039], SAMM: [0.7039795853691472]


In [4]:
class Config(core.Config):
    device = torch.device("cuda:1")
    scheduler = None
    num_workers = 0
    criterion = nn.CrossEntropyLoss
    evaluation_fn = utils.MultiClassF1Score
    epochs = 3000
    model = partial(models.OffApexNet, num_classes=3)
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    early_stopping = True

In [21]:
#offpaexnet
out = EarlyStoppingValidation(Config).validate(df, data[:, :2])

100%|███████████████████████████████████████| 3000/3000 [04:37<00:00, 10.81it/s]


Subject: s1, n=6 | train_mean: 1.0 | test_mean: 0.2778


100%|███████████████████████████████████████| 3000/3000 [04:37<00:00, 10.82it/s]


Subject: s11, n=7 | train_mean: 1.0 | test_mean: 1.0


100%|███████████████████████████████████████| 3000/3000 [04:37<00:00, 10.81it/s]


...

Subject: 035, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|███████████████████████████████████████| 3000/3000 [04:16<00:00, 11.71it/s]


Subject: 036, n=1 | train_mean: 1.0 | test_mean: 1.0


100%|███████████████████████████████████████| 3000/3000 [04:16<00:00, 11.68it/s]


Subject: 037, n=1 | train_mean: 1.0 | test_mean: 1.0
Total f1: [0.7031582952815829], SMIC: [0.6831628481820843], CASME2: [0.774359585053456], SAMM: [0.60521178168237]


In [19]:
#offapexnet early stopping
out = EarlyStoppingValidation(Config).validate(df, data[:, :2])

100%|███████████████████████████████████████| 3000/3000 [04:41<00:00, 10.65it/s]


Subject: s1, n=6 | train_mean: 1.0 | test_mean: 0.4127


100%|███████████████████████████████████████| 3000/3000 [04:41<00:00, 10.67it/s]


Subject: s11, n=7 | train_mean: 1.0 | test_mean: 1.0


100%|███████████████████████████████████████| 3000/3000 [04:46<00:00, 10.47it/s]


...

Subject: 035, n=8 | train_mean: 1.0 | test_mean: 1.0


100%|███████████████████████████████████████| 3000/3000 [04:49<00:00, 10.37it/s]


Subject: 036, n=1 | train_mean: 1.0 | test_mean: 1.0


100%|███████████████████████████████████████| 3000/3000 [04:50<00:00, 10.32it/s]


Subject: 037, n=1 | train_mean: 1.0 | test_mean: 1.0
Total f1: [0.8389628241464461], SMIC: [0.7817601651308129], CASME2: [0.9282855436081244], SAMM: [0.7897925416825761]
